<a href="https://colab.research.google.com/github/yoohw0317/2021knubootcamp/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multivariate Linear Regression**

여러개의 정보를 통해서 하나의 결과를 도출해내는 학습법



matmul()로 한번에 계산
더 간결하고, x의의 길이가 바뀌어도 코드를 바꿀 필요가 없고 속도도 더 빠름.

cost function의 수식은 비슷함.

W : = W -a * d/dx a


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [27]:
# 데이터 (여기가 simple regression과 달라짐)
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화 (여기도 약간 달라짐)
W = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

In [30]:
nb_epochs = 20
for epoch in range(nb_epochs+1):

  #H(x)를 계산)
  hypothesis = x_train.matmul(W) + b

  #Cost 계산, Cost는 점점 작아지도록 됨
  cost = torch.mean((hypothesis - y_train) ** 2)

  #cost로 H(x)를 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print(f'Epoch {epoch:4d}/{nb_epochs} hypothesis: {hypothesis.squeeze().detach()} Cost: {cost.item():.6f}')

Epoch    0/20 hypothesis: tensor([152.8021, 183.6749, 180.9686, 197.0709, 140.1017]) Cost: 1.617638
Epoch    1/20 hypothesis: tensor([152.8019, 183.6754, 180.9687, 197.0710, 140.1022]) Cost: 1.616936
Epoch    2/20 hypothesis: tensor([152.8016, 183.6758, 180.9687, 197.0710, 140.1027]) Cost: 1.616214
Epoch    3/20 hypothesis: tensor([152.8012, 183.6762, 180.9686, 197.0710, 140.1032]) Cost: 1.615513
Epoch    4/20 hypothesis: tensor([152.8008, 183.6765, 180.9686, 197.0710, 140.1037]) Cost: 1.614808
Epoch    5/20 hypothesis: tensor([152.8004, 183.6768, 180.9684, 197.0709, 140.1041]) Cost: 1.614109
Epoch    6/20 hypothesis: tensor([152.8000, 183.6772, 180.9683, 197.0708, 140.1045]) Cost: 1.613405
Epoch    7/20 hypothesis: tensor([152.7995, 183.6775, 180.9682, 197.0706, 140.1049]) Cost: 1.612695
Epoch    8/20 hypothesis: tensor([152.7991, 183.6778, 180.9681, 197.0705, 140.1053]) Cost: 1.611981
Epoch    9/20 hypothesis: tensor([152.7987, 183.6781, 180.9679, 197.0704, 140.1057]) Cost: 1.611289


# nn.Module 사용하기

* nn.Module 을 상속해서 모델을 생성
* nn.Linear (3, 1)
  * 입력 차원: 3
  * 출력 차원: 1
* Hypothesis 계산은 forward()에서 처리됨
* Gradient 계산은 PyTorch가 알아서 해줌 (backward() 통해서)



In [3]:
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

# F.mse_loss


*   torch.nn.functional 에서 제공하는 loss function 사용
*   쉽게 다른 loss와 교체 가능 (l1_loss, smooth_l1_loss 등등)



In [9]:
# 데이터 (위의 코드와 변화 없음)
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화 - 매우 간결해짐
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer_new = optim.SGD(model.parameters(), lr=1e-5)

In [10]:
nb_epochs = 50
for epoch in range(nb_epochs+1):

  #H(x)를 계산 - model을 이용해서 계산하게 됨.
  Hypothesis_new = model(x_train)

  #Cost 계산, Cost는 점점 작아지도록 됨
  cost_new = F.mse_loss(Hypothesis_new, y_train)

  #cost로 H(x)를 개선
  optimizer_new.zero_grad()
  cost_new.backward()
  optimizer_new.step()

  print(f'Epoch {epoch:4d}/{nb_epochs} hypothesis: {Hypothesis_new.squeeze().detach()} Cost: {cost_new.item():.6f}')

Epoch    0/50 hypothesis: tensor([-10.4987,  -6.2474,  -9.4404, -10.5937,  -2.8628]) Cost: 32507.056641
Epoch    1/50 hypothesis: tensor([59.9116, 78.3796, 73.9445, 80.2103, 61.6860]) Cost: 10190.708008
Epoch    2/50 hypothesis: tensor([ 99.3320, 125.7590, 120.6287, 131.0482,  97.8243]) Cost: 3195.718750
Epoch    3/50 hypothesis: tensor([121.4023, 152.2848, 146.7656, 159.5105, 118.0566]) Cost: 1003.160339
Epoch    4/50 hypothesis: tensor([133.7589, 167.1354, 161.3987, 175.4455, 129.3837]) Cost: 315.908936
Epoch    5/50 hypothesis: tensor([140.6771, 175.4496, 169.5914, 184.3671, 135.7252]) Cost: 100.491470
Epoch    6/50 hypothesis: tensor([144.5506, 180.1042, 174.1782, 189.3619, 139.2753]) Cost: 32.968658
Epoch    7/50 hypothesis: tensor([146.7195, 182.7100, 176.7463, 192.1584, 141.2626]) Cost: 11.803207
Epoch    8/50 hypothesis: tensor([147.9340, 184.1687, 178.1841, 193.7241, 142.3750]) Cost: 5.168192
Epoch    9/50 hypothesis: tensor([148.6142, 184.9852, 178.9892, 194.6008, 142.9977]) 